# 1. Fine tunning LLaMA2  V2 With Adapters / QA Text2Sql


In [1]:
#!pip install fsspec==2024.10.0
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes

#! pip install -q -U rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [2]:
! pip install torchvision==0.20.1+cu124 torch==2.5.1 torchaudio==2.5.1+cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.29.post2 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [6]:
!pip install -q -U huggingface_hub
# In case Login Required For Model
from huggingface_hub import login

### Imports

In [24]:
import pandas as pd
import json
import torch
import os
import time

# In case Login Required For Model
# from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print
from langchain.llms import HuggingFacePipeline

In [ ]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
# os.environ["TOKENIZERS_PARALLELISM"]="false"
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

# first let's load the dataset , model and it 's tokenizer

In [70]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `finetoken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `finetoke

In [ ]:
hf_qrCpVcbPGcqpTurxIZOUkGjjhqBxpbLEDMmodel_id = "TheBloke/Llama-2-7B-GPTQ"
# model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"
MODEL_PATH = "/content/drive/MyDrive/Text2SQL/sql_coder/"
MODEL_DIR = "sql_gptq_training"+"_80/"
checkpoint_name ="SQL_gpt_V2"+time.strftime("%Y%m%d_%H%M%S")
OUT_DIR = MODEL_PATH + MODEL_DIR + checkpoint_name
print(OUT_DIR)
#OUT_DIR ="/content/drive/MyDrive/sql_gptq_training"  # Save to Google Drive

/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642

In [10]:
print(checkpoint_name)

SQL_gpt_V220250208_043642

In [11]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"

In [12]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

In [13]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

### Load and Check Data

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Load Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [18]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [19]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [20]:
display(test_df.head(3))
display(test_df.shape)

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...


(454, 5)

### lets load the  Model with Quantization and Tokenizer

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# LLM GPTQ Model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config_loading,
                                             device_map="auto")

### Let's define LORA and other configration

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
# %%
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,  # Increase rank for better adaptation
    lora_alpha=64,  # Higher alpha to scale weight updates
    target_modules=["k_proj", "o_proj", "q_proj", "v_proj"],
    lora_dropout=0.1,  # Slightly higher dropout for regularization
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.use_cache = False # silence the warnings. Please re-enable for inference!

trainable params: 8,388,608 || all params: 270,798,848 || trainable%: 3.0977


In [ ]:
args = TrainingArguments(
    per_device_train_batch_size=8,  # Increase batch size (may require more GPU memory)
    gradient_accumulation_steps=2,  # Less accumulation, more frequent updates
    warmup_steps=10,  # Longer warmup to stabilize learning
    num_train_epochs=3,  # More epochs for stronger training
    learning_rate=3e-4,  # Higher LR for faster adaptation
    fp16=True,  # Use mixed precision training
    logging_steps=10,
    output_dir=OUT_DIR,
    overwrite_output_dir=True,
    optim="adamw_torch",  # Try PyTorch's AdamW optimizer
    save_strategy="epoch",
    evaluation_strategy="epoch",
    report_to="none",
    save_total_limit=2
)

In [ ]:
# set training arguments - Feel free to adapt it
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    warmup_steps=2,
    max_steps=510, #100, # Change this if T4 Timesout
    optim="adamw_hf",
    learning_rate=2e-4,
    fp16=True, #use mixed precision training
    # predict_with_generate=True, # Needed for Seq2Seq models only
    logging_steps=1, #500,
    save_strategy="epoch",
    #save_steps=1000,
    #eval_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False, #True
    report_to="none"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args, # Training Only
    # args=training_args, # Evaluation
    # compute_metrics=compute_metrics, # uncomment for ROGUE Metrics
    train_dataset=dataset['train'],
    # eval_dataset = dataset['test'], # Evaluation Only
    peft_config=config,
    #dataset_text_field="text",
    tokenizer=tokenizer,
    #packing=False,
    #max_seq_length=512
    )

In [54]:
# Takes ~hours to finetune
train_result = trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,0.413300
2,0.395900
3,0.426200
4,0.440000
5,0.479200
6,0.423700
7,0.485100
8,0.445000
9,0.477200
10,0.457200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


In [55]:
#output_dir = os.path.join(args.output_dir, checkpoint_name)
#trainer.model.save_pretrained(output_dir)
output_dir = OUT_DIR  # OUT_DIR already includes the full path
print("Saving model to:", output_dir)

# Save the model
trainer.model.save_pretrained(output_dir)

Saving model to: /content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642

### Now let's perform inference on an Example of the Test dataset

In [ ]:
# To perform inference on the test dataset example load the model from the checkpoint
persisted_model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
)
persisted_model.config.use_cache = True

In [61]:
ID = 33
print('Question:')
display(dataset['test'][ID]['question'])
print('Context:')
display(dataset['test'][ID]['context'])
print('Response:')
display(dataset['test'][ID]['response'])

Question:

'List the record company shared by the most number of orchestras.'

Context:

'CREATE TABLE orchestra (Record_Company VARCHAR)'

Response:

'SELECT Record_Company FROM orchestra GROUP BY Record_Company ORDER BY COUNT(*) DESC LIMIT 1'

In [62]:
text = test_df['text'][ID]
print(text)

### Instruction:
    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a
question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    List the record company shared by the most number of orchestras.
    ### Context:
    CREATE TABLE orchestra (Record_Company VARCHAR)
    ### Response:

In [ ]:

text = test_df['text'][ID]
inputs = tokenizer(text, return_tensors="pt").to('cuda')
generation_config = GenerationConfig(
    penalty_alpha=0.5,
    # do_sample = True,
    top_k=1,
    # temperature=0.1,
    repetition_penalty=1.2,
    max_new_tokens=180,
    use_cache=True,

)
start_time = perf_counter()
outputs = persisted_model.generate(**inputs, generation_config=generation_config)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
output = parse(tokenizer.decode(outputs[0]))
result = {'response': output}
print(json.dumps(result))
end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

In [66]:
# display(test_df.head(2))
print('Data:  ' + test_df.loc[ID, 'response'])
print('LLM_:  ' + output)

Data:  SELECT Record_Company FROM orchestra GROUP BY Record_Company ORDER BY COUNT(*) DESC LIMIT 1

LLM_:  SELECT Record_Company FROM orchestra GROUP BY Record_Company ORDER BY COUNT(*) DESC LIMIT 1

# Push the Model to Hugging Face

In [64]:
# Save merged model (or non-PEFT model)
# Merge LoRA into the base model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642/tokenizer_config.json',
 '/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642/special_tokens_map.json',
 '/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642/tokenizer.model',
 '/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642/added_tokens.json',
 '/content/drive/MyDrive/Text2SQL/sql_coder/sql_gptq_training_80/SQL_gpt_V220250208_043642/tokenizer.json')

In [ ]:
repo_name = "khalifa1/Sql_LLama2_V3"

In [71]:
# !huggingface-cli login
#model.push_to_hub(new_model, use_temp_dir=False)
#tokenizer.push_to_hub(new_model, use_temp_dir=False)

model.push_to_hub(repo_id=repo_name,use_temp_dir=True)
tokenizer.push_to_hub( repo_id=repo_name,use_temp_dir=True)

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/khalifa1/Sql_gpt_turbo/commit/8fdd3ceb9ee84f046657c13c64e485a92545873e', commit_message='Upload tokenizer', commit_description='', oid='8fdd3ceb9ee84f046657c13c64e485a92545873e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/khalifa1/Sql_gpt_turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='khalifa1/Sql_gpt_turbo'), pr_revision=None, pr_num=None)